In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import pymongo
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk

In [2]:
# driver_genius = webdriver.Chrome(ChromeDriverManager().install())
# ouvrir le site et accepter les cookies
# driver_genius.get("https://genius.com/Miley-cyrus-flowers-lyrics")
# driver_genius.find_element(By.XPATH,"/html/body/div[2]/div[3]/div[1]/div/div[2]/div/button[1]").click()
# driver_genius.find_element(By.CSS_SELECTOR,"SongInfo__Title-nekw6x-1 iRKrFW").text
# for i in range(len(genius)):

#     # activer la recherche
#     element = driver_genius.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/form/input")
#     element.send_keys(genius[i][0]+' '+genius[i][1], Keys.ENTER)


#     # clicker sur la chanson
#     driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a/div[2]").click()
#     # date = driver_genius.find_element(By.XPATH,"/html/body/div[1]/main/div[1]/div[4]/div/div[1]/div[2]/div")
#     # print(date.text)
#     # print(driver_genius.find_element(By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div[1]/div[2]/div").text)
#     print(i)
#     driver_genius.get("https://genius.com/")
#     # on revient à l'accueil
#     driver_genius.back()
#     driver_genius.back()


#     WebDriverWait(driver_genius, timeout=10)
# time.sleep(100)


In [3]:
df = pd.read_csv('dataframe.csv',delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            2400 non-null   object
 1   Artist           2400 non-null   object
 2   Rank             2400 non-null   int64 
 3   Last Week        2400 non-null   object
 4   Peak Positon     2400 non-null   int64 
 5   Weeks on charts  2400 non-null   int64 
 6   Date             2400 non-null   object
dtypes: int64(3), object(4)
memory usage: 131.4+ KB


In [4]:
data = df.to_dict('records')
# data = df.to_json(date_format='dd/mm/yyyy',orient='records')
# data

In [5]:
# client = MongoClient("mongodb://localhost:27017/")
# db = client["mydatabase"]
# collection = db["mycollection"]

# print(client.list_database_names())
# print(db.list_collection_names())
# client.drop_database(db)
# collection.drop()
# print(client.list_database_names())
# print(db.list_collection_names())

# db = client["mydatabase"]
# collection = db["mycollection"]
# print(type(client))
# print(type(db))
# print(collection)

# collection.insert_many(data)
# collection.find_one()

# cursor = collection.find()
# # next(cursor)
# for document in cursor :
#     # print('-----')
#     print(document)

In [6]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,"scheme": "http"}])

In [7]:
es.ping()

True

In [8]:
# document = {"Title":"Anti-Hero","Artist":"Taylor Swift","Rank":1,"Last Week":"2","Peak Positon":1,"Weeks on charts":4,"Date":"26\\/11\\/2022"}

In [9]:
# response = es.index(index="billboard", id=1 ,document=document)
# response['result']
# response

In [10]:
# res = es.get(index="billboard",id=1)
# # pprint.pprint(res["_source"])
# res["_source"]

In [11]:
# # define the index name and document type
# index_name = 'billboard'
# doc_type = '_doc'

# # define the action list to be passed to the bulk method
# actions = [
#     {
#         '_index': index_name,
#         '_type': doc_type,
#         '_source': data
#     }
#     for i in data
# ]

# # use the bulk method to index all documents
# response = bulk(es, actions=actions)

In [12]:
def generate_data(data):
    for docu in data:
        yield {
            "_index": "billboard",
            "_type": "song",
            "_source": {k:v if v else None for k,v in docu.items()},
        }

bulk(es, generate_data(data))

c:\Users\camil\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


(2400, [])

In [20]:
result = es.search(index="billboard", query= {"match_all": {}})
result

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2400, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'billboard',
    '_type': 'song',
    '_id': 'wasuW4YBxD1DNUC-qFhN',
    '_score': 1.0,
    '_source': {'Title': 'Anti-Hero',
     'Artist': 'Taylor Swift',
     'Rank': 1,
     'Last Week': '2',
     'Peak Positon': 1,
     'Weeks on charts': 4,
     'Date': '26/11/2022'}},
   {'_index': 'billboard',
    '_type': 'song',
    '_id': 'wqsuW4YBxD1DNUC-qFhN',
    '_score': 1.0,
    '_source': {'Title': 'Unholy',
     'Artist': 'Sam Smith & Kim Petras',
     'Rank': 2,
     'Last Week': '4',
     'Peak Positon': 1,
     'Weeks on charts': 8,
     'Date': '26/11/2022'}},
   {'_index': 'billboard',
    '_type': 'song',
    '_id': 'w6suW4YBxD1DNUC-qFhN',
    '_score': 1.0,
    '_source': {'Title': 'Rich Flex',
     'Artist': 'Drake & 21 Savage',
     'Rank': 3,
     'Last Week': '1',
     'Peak Po

In [21]:
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

'2400 documents correspondent à la requêtes qui a pris 0 ms'

In [14]:
!curl http://localhost:9200/_cat/indices?v #!curl http://elasticsearch:9200/_cat/indices?v

health status index                    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .kibana_task_manager_1   EXIwN7TRSVqNquk2ayLb4A   1   0          2            1     16.8kb         16.8kb
green  open   .apm-agent-configuration UL5dxL77QXCnt2-BKIT2aQ   1   0          0            0       283b           283b
yellow open   billboard                2vrh2HRMRUWtqJwXYWhJ6w   1   1          0            0    128.1kb        128.1kb
green  open   .kibana_1                jodusnl1QCuev0azrBTcxg   1   0         19            8       79kb           79kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   600  100   600    0     0  54107      0 --:--:-- --:--:-- --:--:-- 60000
curl: (3) URL using bad/illegal format or missing URL
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0curl: (6) Could not resolve host: elasticsearch


In [15]:
!curl http://localhost:9200/billboard/_search #!curl http://elasticsearch:9200/movies/_search

{"took":0,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":2400,"relation":"eq"},"max_score":1.0,"hits":[{"_index":"billboard","_type":"song","_id":"wasuW4YBxD1DNUC-qFhN","_score":1.0,"_source":{"Title":"Anti-Hero","Artist":"Taylor Swift","Rank":1,"Last Week":"2","Peak Positon":1,"Weeks on charts":4,"Date":"26/11/2022"}},{"_index":"billboard","_type":"song","_id":"wqsuW4YBxD1DNUC-qFhN","_score":1.0,"_source":{"Title":"Unholy","Artist":"Sam Smith & Kim Petras","Rank":2,"Last Week":"4","Peak Positon":1,"Weeks on charts":8,"Date":"26/11/2022"}},{"_index":"billboard","_type":"song","_id":"w6suW4YBxD1DNUC-qFhN","_score":1.0,"_source":{"Title":"Rich Flex","Artist":"Drake & 21 Savage","Rank":3,"Last Week":"1","Peak Positon":1,"Weeks on charts":2,"Date":"26/11/2022"}},{"_index":"billboard","_type":"song","_id":"xKsuW4YBxD1DNUC-qFhN","_score":1.0,"_source":{"Title":"I'm Good (Blue)","Artist":"David Guetta & Bebe Rexha","Rank":4,"Last Week":"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2406  100  2406    0     0   453k      0 --:--:-- --:--:-- --:--:--  587k
curl: (3) URL using bad/illegal format or missing URL
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: elasticsearch


In [16]:
# res = es.index(index="algorithms", doc_type='billboard', id=1, body=data)
# print(res['result'])

In [17]:
# Define the Elasticsearch document
class MyDocument(Document):
    title = Text()
    artist = Text()
    rank = Integer()
    class Meta:
        index = "your_index"


In [18]:
# # Transfer the data from MongoDB to Elasticsearch
# for document in collection.find():
#     doc = MyDocument(meta={"id": document["_id"]}, title=document["Title"], artist=document["Artist"],rank=document["Rank"])
#     doc.save(using=es)